In [1]:
from utils.load_data import *
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from hampel import hampel

In [4]:
df = pd.read_csv("../room2/idle2.csv")

df_csi = df.loc[:, ['len', 'CSI_DATA']]
drop_idx = []
for i in range(df_csi.shape[0]):
    if df_csi.iloc[i]['len'] < 384:
      drop_idx.append(i)

df_csi = df_csi.drop(drop_idx)
size_x = len(df_csi.index)
size_y = df_csi.iloc[0]['len']//2 # no. of subcarriers ..

array_csi = np.zeros([size_x, size_y], dtype = np.complex64)

for x , csi in enumerate(df_csi.iloc):
  temp = csi["CSI_DATA"].replace(' ', ',')
  if(temp[-1]!= "]"):
      temp+="]"
  else:
    temp = temp.replace(',]', ']')
  csi_raw_data = json.loads(temp)
  for y in range(0, len(csi_raw_data)-1, 2):
      array_csi[x][y//2] = complex(csi_raw_data[y], csi_raw_data[y + 1])  # IQ channel frequency response
array_csi_modulus = abs(array_csi) 

JSONDecodeError: Expecting value: line 1 column 477 (char 476)

In [3]:
still1 = load_csi("../room1/still1.csv")
still2 = load_csi("../room1/still2.csv")
still3 = load_csi("../room2/idle1.csv")
#still4 = load_csi("../room2/idle2.csv")
walk1 = load_csi("../room1/walk1.csv")
walk2 = load_csi("../room1/walk2.csv")
walk3 = load_csi("../room2/walk1.csv")
walk4 = load_csi("../room2/walk2.csv")

JSONDecodeError: Expecting value: line 1 column 218 (char 217)

In [6]:
temp = csi["CSI_DATA"].replace(' ', ',')
temp

'[110,96,6,0,0,0,0,0,0,0,0,0,-6,24,-6,22,-7,26,-10,26,-9,24,-9,20,-10,22,-10,20,-9,21,-11,20,-13,22,-12,22,-12,21,-13,19,-11,16,-11,21,-10,17,-11,16,-10,16,-9,15,-10,15,-10,16,-11,13,-10,15,-11,16,-7,15,0,0,-8,15,-6,16,-6,15,-5,14,-6,13,-5,13,-5,14,-4,12,-4,13,-2,12,-1,13,-2,11,-2,12,0,11,0,11,1,11,1,12,2,10,2,10,3,8,1,10,2,9,2,9,2,9,2,8,2,8,0,0,0,0,0,0,0,0,0,0,-1,-1,-1,-1,3,-13,2,-13,1,-12,-2,-11,3,-9,2,-8,2,-8,1,-8,0,-6,-1,-6,-2,-6,-2,-5,-3,-6,-4,-5,-4,-5,-5,-4,-6,-4,-6,-'

In [4]:
def hampel_smooth(time_series,axis = 1):
    ham_series  = time_series.copy()
    for sc in range(time_series.shape[axis]):
        #print(sc)
        sc_ts = pd.Series(time_series[:,sc])
        #print(sc_ts.shape)
        ham_series[:,sc] = hampel(sc_ts, window_size=10, n=3, imputation=True)
        
    return ham_series

In [5]:
walk1_hampel = hampel_smooth(walk1)
walk2_hampel = hampel_smooth(walk2)
still1_hampel = hampel_smooth(still1)
still2_hampel = hampel_smooth(still2)

In [6]:
still = np.append(still1_hampel,still2_hampel,axis=0)
walk = np.append(walk1_hampel,walk2_hampel,axis=0)

In [7]:
pca = PCA(n_components=6)
still_pca = pca.fit_transform(still)
pca = PCA(n_components=6)
walk_pca = pca.fit_transform(walk)

In [8]:
def window_conv(data, stride=4, window=10):   
    x = []
    for i in range(0,len(data)-window,stride):
        x.append(data[i:i+window])
    return np.array(x)

In [9]:
x1 = window_conv(still_pca)
x2 = window_conv(walk_pca)
x = np.append(x1,x2,axis=0)
y = np.append(np.zeros(len(x1)),np.ones(len(x2)))

In [10]:
np.random.seed(54)
np.random.shuffle(x)
np.random.shuffle(y)

In [11]:
x = x[:,:,1:]
x = x.reshape(x.shape[0],-1)

In [12]:
x_train,x_test = x[:2000], x[2000:]
y_train,y_test = y[:2000], y[2000:]

In [13]:
x.shape

(2692, 50)

In [14]:
import keras
from keras.models import Sequential
from keras.layers import Dense# Neural network
model = Sequential()
model.add(Dense(10, input_dim=30, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(2, activation='relu'))

In [15]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs=100, batch_size=64)